# Library

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import pprint
import time

: 

# Database

In [ ]:
client = MongoClient('203.255.92.141:27017', connect=False)
ID = client['ID']
ID_Domestic = ID['Domestic']
ntis_client  = client['NTIS']
ntis_rawdata = ntis_client['Rawdata']
scienceon = client['SCIENCEON']
scienceon_authorpapers = scienceon['AuthorPapers']
scienceon_rawdata = scienceon['Rawdata']

# Program

In [ ]:
scienceon_id = []
papers_content = []
scienceon_dict = {}
ntis_data = []

In [ ]:
#NTIS author ID and papers
AuthorPapers_A_ID_AND_Papers = list(ntis_client['AuthorPapers'].find({'keyId':519}, {'A_ID':1,"papers":1}))
for A_ID_Data in AuthorPapers_A_ID_AND_Papers[0:46]:
    scienceon_author = []
    #scienceon_data = []
    scienceon_dict['ntis'] = A_ID_Data['A_ID'] #NTIS ID
    paper_cnt = [] #Number of papers
    paper_cnt.append(len(A_ID_Data['papers']))
    print(paper_cnt)
    papers_content = []
    # keyid 519로 검색했을 경우의 논문
    for id_paper in A_ID_Data['papers']: #NTIS Papers content
        ntis_paper_data = list(ntis_rawdata.find({"_id":id_paper},{"koTitle":1,"enTitle":1,"absAbs":1,"effAbs":1,"koKeyword":1,"enKeyword":1,"goalAbs":1,
        'totalFund':1,"mngId":1,"prdEnd":1,"prdStart":1}))
        papers_content.append(ntis_paper_data[0])
    #print(scienceon_dict)
    
    # keyid 519로 검색했을 경우 나오는 a_id에 대한 scienceon의 id
    author = ID_Domestic.find({"ntis":A_ID_Data['A_ID']},{"scienceon":1,"ntis":1})
    for id in author:
        for scienceon_id in id["scienceon"]:
            #paper_cnt.append(len(A_ID_Data['papers']))
            scienceon_author.append(scienceon_id) #SCIENCEON ID
            #print(scienceon_author)
            a = scienceon_authorpapers.find({"A_ID":scienceon_id, "keyId":519},{"papers":1})
            if a:
                paper_cnt.append(0)
            else:
                for scienceon_paper_id in a: #Scienceon Papers content
                    print(scienceon_paper_id['papers'])
                    #print(len(scienceon_paper_id['papers']))
                    #print(scienceon_id)
                    paper_cnt.append(len(scienceon_paper_id['papers']))
                    scienceon_paper_data = list(scienceon_rawdata.find({"_id":scienceon_paper_id['papers'][0]},{"title":1,"abstract":1,"english_title":1,"qryKeyword":1,"issueInsts":1,
                'issueLangs':1,'Usage Count':1,'issue_year':1,'citation':1,'author_id':1,"paper_keyword":1,"english_title":1,"english_abstract":1,"author_inst":1,"issue_inst":1,
                "issue_lang":1}))
                    papers_content.append(scienceon_paper_data[0])
        #print(scienceon_author)     
    scienceon_dict['scienceon'] = scienceon_author
    scienceon_dict['papers_number'] = paper_cnt
    scienceon_dict['papers'] = papers_content
    print(scienceon_dict)    